In [6]:
# Here is my artillery Shot data calculations with coriolis removed because it is so small

from numpy import radians, sin, exp, cos, tan, sqrt, arctan, pi, log
import matplotlib.pyplot as plt
import pandas as pd

# Constants
gp = 9.832
# Inputs
# Example usage:
v0 = 790 #m/s
launch_angle = 331.9/3200*180 #mils to degrees
mass = 103.5/2.2 # lbs to kg
drag_coefficient = .25
radius = .155/2
area = radius**2*pi
latitude = 32 + 3/60 + 31/3600 # Frankfurt's approximate latitude
azimuth = 0/3200*180 # Firing due east
initial_altitude = 1379 # Launch altitude in meters
target_altitude = 2094 # Target altitude in meters
rho0 = 1.225 #float(input("Enter the air density at altitude (kg/m^3): "))
# Additional input for Magnus effect
spin_rate = v0/(20*.155) #float(input("Enter the projectile spin rate (revolutions per second): ")
spin_velocity = spin_rate * 2 * pi  # Convert from rev/s to rad/s


# Function to calculate gravitational acceleration with latitude
def gravity_with_latitude(latitude, gp=gp, alpha=0.0053024):
    latitude_rad = radians(latitude)
    return gp * (1 - alpha * cos(latitude_rad)**2)

# Function to calculate air density with altitude
def air_density_with_altitude(h, rho0=rho0, H=8500):
    return rho0 * exp(-h / H)

# Constants for Magnus effect
S = 0.0009 # This is an example value; you'll need to determine the appropriate value for your projectile

# Function to calculate the Magnus force
def magnus_force(v, spin_velocity, rho, area, S=S):
    # CL is the lift coefficient, which is dimensionless
    # spin_rate is in rad/s to be compatible with the velocity v in m/s
    CL = S * (spin_velocity / v)
    # Magnus force calculation, with v in m/s and area in m^2
    return CL * 0.5 * rho * area * v**2

# Function to calculate projectile motion with drag, varying air density, varying gravity
def projectile_motion_with_drag(v0, angle, mass, drag_coefficient, area, latitude, azimuth, initial_altitude, target_altitude):
    # Convert angle to radians
    angle_rad = radians(angle)
    
    # Initial velocity components
    v0x = v0 * cos(angle_rad)
    v0y = v0 * sin(angle_rad)
    # Velocity components
    vx = v0x
    vy = v0y
    vz = 0
    
    # Time step for simulation
    dt = 0.01
    t = 0
    x = 0
    y = initial_altitude  # Set the initial altitude
    z = 0  # z-axis for lateral drift
    
    # Lists to store the trajectory points
    t_time = []
    x_points = []
    y_points = []
    z_points = []  # z-axis for lateral drift
    vx_comp = []
    vy_comp = []
    vz_comp = []
    v_mag = []
    qe = []

    # Store the initial points
    t_time.append(t) # seconds
    x_points.append(x) # meters
    y_points.append(y) # meters
    z_points.append(z)  # z-axis for lateral drift
    vx_comp.append(v0x) # m/s
    vy_comp.append(v0y) # m/s
    vz_comp.append(0) # m/s
    v_mag.append(v0) # m/s
    qe.append(angle/180*3200) # trajectory in mils

    
    
    # Calculate the gravitational acceleration for the given latitude
    g = gravity_with_latitude(latitude)
    
    # Simulation loop
    while y >= 0:
        # Calculate the air density at current altitude
        rho = air_density_with_altitude(y)

        # Calculate the Magnus force
        # The velocity v is the magnitude of the velocity vector, in m/s
        F_magnus = magnus_force(sqrt(vx**2 + vy**2), spin_velocity, rho, area)
        
        # Calculate the drag force
        F_drag_x = - (drag_coefficient * rho * area * vx**2) / (2 * mass)
        F_drag_y = - (drag_coefficient * rho * area * vy**2) / (2 * mass)
        F_drag_z = - (drag_coefficient * rho * area * vz**2) / (2 * mass)
        F_total_z = F_magnus + F_drag_z
                   
        # Update velocities
        vx += F_drag_x * dt
        vy += (F_drag_y - g) * dt
        vz += F_total_z * dt / mass
        
        # Update positions
        t += dt
        x += vx * dt
        y += vy * dt
        z += vz * dt
        
        # Store the points
        t_time.append(t) # seconds
        x_points.append(x)
        y_points.append(y)
        z_points.append(z)  # z-axis for lateral drift
        vx_comp.append(vx) #m/s
        vy_comp.append(vy)# m/s
        vz_comp.append(vz)
        v_mag.append(sqrt(vx**2+vy**2)) # m/s
        qe.append(arctan(vy/vx)/pi*3200) #in mils
        
        # Check if the projectile has reached the target altitude
        if y <= target_altitude and vy < 0:
            break
        
                  
    return t, x, y, z, t_time,x_points, y_points, z_points,vx_comp,vy_comp,vz_comp,v_mag,qe

# Call the function and display the results
time_of_flight, final_x, final_y, final_z, t_time, x_vals, y_vals, z_vals, vx_comp, vy_comp,vz_comp, v_mag,qe = projectile_motion_with_drag(v0, launch_angle, mass, drag_coefficient, area, latitude, azimuth, initial_altitude, target_altitude)

shotdata = pd.DataFrame({'t':t_time,'x':x_vals,'y':y_vals,'z':z_vals, 'vx':vx_comp, 'vy':vy_comp,'vz':vz_comp, 'QE':qe,'v_mag':v_mag})
shotdata2 = shotdata
shotdata2['t2'] = shotdata2['t'] * (v0**2*8.3784685932988*10**-7-0.00120211574427114*v0+1.26318480878614)
shotdata2['x2'] = shotdata['x'] * (v0**2*1.2984339385335*10**-6-0.00187546325673901*v0+1.45874083097553)
shotdata2['y2'] = shotdata['y'] * (v0**2*1.2282602669966*10**-6-0.00184705039255366*v0+1.38581268739332)
shotdata2['drift'] = arctan(shotdata['z']/shotdata['x'])/pi*3200*(v0**2*2.0447751577913*10**-5-0.0294119701879176*v0+12.2795878968585)
shotdata2['z2'] = shotdata2['x2']*tan(shotdata2['drift']/3200*pi)
shotdata2['dx'] =  shotdata2['x2'].diff()
shotdata2['dy'] =  shotdata2['y2'].diff()
shotdata2['dt'] =  shotdata2['t2'].diff()
shotdata2['qe2'] = arctan(shotdata2['dy']/shotdata2['dx'])/pi*3200
shotdata2['v2'] = sqrt(shotdata2['dy']**2+shotdata2['dx']**2)/shotdata2['dt']

shotdata2.iloc[:,9:].to_csv('afatds_shotdata.csv')

# Note: The lateral drift is represented in the z-axis, which is not plotted in this 2D graph.
print(f"Time of Flight until reaching target altitude: {max(shotdata2['t2']):.2f} seconds")
print(f"Horizontal Distance traveled until reaching target altitude: {max(shotdata2['x2']):.2f} meters")
print(f"Max Ord: {max(shotdata2['y2']):.2f} meters")
print(f"Drift: {max(shotdata2['z2'][-5:]):.2f} meters and  {max(shotdata2['drift'][-5:]):.2f} mils")
print(f"Angle of Fall: {max(shotdata2['qe2'][-5:]):.2f} mils")
print(f"Terminal Velocity: {max(shotdata2['v2'][-5:]):.2f} m/s")

# Plotting the trajectory
plt.figure(figsize=(10, 5))
plt.plot(shotdata2['x2'],shotdata2['y2'], label='Projectile Trajectory')
plt.title('Projectile Motion with Drag, Varying Air Density, Varying Gravity, Initial and Target Altitude')
plt.xlabel('Distance (m)')
plt.ylabel('Height (m)')
plt.legend()
plt.grid(True)
plt.show()

shotdata2.iloc[:,9:]


ModuleNotFoundError: No module named 'matplotlib'